In [84]:
import os 
from torch.utils.data import DataLoader, Dataset
from PIL import Image

import torch
from torch import nn
from torch.nn import functional as F


In [85]:
class Loader(Dataset):
    def __init__(self, split, trasform = None):
        base_dir='./datasets/'

        path =os.path.join(base_dir,'64x64_SIGNS/{}'.format(split))

        files = os.listdir(path)

        self.filenames = [os.path.join(path,f) for f in files if f.endswith('.jpg')]

        self.targets = [int(f[0]) for f in files]

        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx])

        if self.transform:
            image = self.transform(image)
        
        return image, self.targets[idx]

In [86]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [87]:
trainSet = Loader('train_signs',transform)
trainLoader = DataLoader(trainSet, batch_size=2)

valSet = Loader('val_signs',transform)
valLoader = DataLoader(valSet, batch_size=1)

testSet = Loader('test_signs',transform)
testLoader = DataLoader(testSet, batch_size=35)

dataLoader = {'train':trainLoader,'val':valLoader,'test':testLoader}

In [88]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=9,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=9,out_channels=12,kernel_size=5)
        self.fc1 = nn.Linear(2028,1000)
        self.fc2 = nn.Linear(1000,500)
        self.fc3 = nn.Linear(500,6)
        self.pool = nn.MaxPool2d(2)

    def forward(self,x):
        x = self.pool(F.relu( self.conv1(x)))
        x = self.pool(F.relu( self.conv2(x)))

        x = x.view(-1,2028)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        x = F.log_softmax(x,dim=1)
        return x


In [89]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [90]:
net = Net()
net.to(device)

Net(
  (conv1): Conv2d(3, 9, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(9, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2028, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=6, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [91]:
from torch import optim 

loss_fn = nn.NLLLoss()

optimizer = optim.Adam(net.parameters(),lr=0.001)

In [92]:
epocs = 100

In [93]:
for epoch in range(epocs):
    

    for inputs, targets in dataLoader['train']:

        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()#after each epoc i going to research for a new best result so i put the gradient on 0 cause i'll start all over again

        outputs = net(inputs)#fitting my images in the network to obtain a result to be used to predict 
        

        loss = loss_fn(outputs, targets) #get the loss

        loss.backward() #calculate the gradients automatically
 
        optimizer.step() #upgrade parametters

        print('loss: ',loss.item(),'      epoc: ',epoch)




        





loss:  0.0009694099426269531       epoc:  9
loss:  0.015195846557617188       epoc:  9
loss:  0.023121356964111328       epoc:  9
loss:  0.0001957416534423828       epoc:  9
loss:  6.341934204101562e-05       epoc:  9
loss:  0.13543909788131714       epoc:  9
loss:  0.0004260540008544922       epoc:  9
loss:  0.19854259490966797       epoc:  9
loss:  3.24249267578125e-05       epoc:  9
loss:  0.0004961490631103516       epoc:  9
loss:  0.0033464431762695312       epoc:  9
loss:  0.012044310569763184       epoc:  9
loss:  0.00033593177795410156       epoc:  9
loss:  0.08618021011352539       epoc:  9
loss:  9.5367431640625e-07       epoc:  9
loss:  0.0       epoc:  9
loss:  3.695487976074219e-05       epoc:  9
loss:  0.00025653839111328125       epoc:  9
loss:  0.0010724067687988281       epoc:  9
loss:  2.5272369384765625e-05       epoc:  9
loss:  0.025109529495239258       epoc:  9
loss:  0.0012431144714355469       epoc:  9
loss:  0.0016155242919921875       epoc:  9
loss:  0.0015310

KeyboardInterrupt: 

In [ ]:
valSet = Loader('val_signs',transform)
image , label = valSet2[20]
image = image.unsqueeze(0)
image = torch.tensor(image,dtype=torch.float32)
image = image.to(device)

In [ ]:
label

In [ ]:
net = net.eval()
out = net(image)
out